In [13]:
class Test:
    __slots__ = ('a',)
    def __init__(self, a):
        self.a = a
    
t = Test(2)
t.a = 3
t.a

3

In [103]:
from dataclasses import dataclass

@dataclass
class X:
    __slots__ = ('a', 'b')
    a: int
    
    def __post_init__(self):
        self.b = 2
    
_ = X(2)
_.a = 3
_.a
_.b = 4
_.b

4

In [105]:
from enum import Enum
from datetime import datetime
from typing import NamedTuple

class Seperator(Enum):
    EMPTY = ''
    HYPHEN = '-'
    UNDERSCORE = '_'


class DatePattern(Enum):
    YYYYMMDD = '%Y%m%d'


class TimePattern(Enum):
    HHmmss = '%H%M%S'


@dataclass
class DatetimeParser:
    __slots__ = ('date', 'time', 'seperator', 'from_beginning')
    date: DatePattern
    time: TimePattern
    seperator: Seperator
    from_beginning: bool  # True = datetime is at the beginning of the filename, False = it's at the end

    def __str__(self) -> str:
        now = datetime.now()
        pattern = self._format()
        string = now.strftime(pattern)
        return string

    def _format(self) -> str:
        pattern = self.date.value + self.seperator.value + self.time.value
        return pattern

    def demo(self):
        return self.__str__()

    def str_to_datetime(self, string) -> datetime:
        pattern = self._format()
        dtime = datetime.strptime(string, pattern)
        return dtime


class Metadata(NamedTuple):
    path: str = None
    filename: str = None
    format: str = None
    starttime: datetime = None
    endtime: datetime = None
    fps: float = None
    width: int = None
    height: int = None
    format: str = None
    recording_token: str = None
    recordingblock_token: str = None
    status: str = None


class VideoFile(NamedTuple):
    __slots__ = ('path', 'metadata_videofile', 'metadata_sidecarfile')
    path: str
    metadata_videofile: Metadata = Metadata()
    metadata_sidecarfile: Metadata = Metadata()

    def metadata_from_videofile(self, datetime_parser: DatetimeParser, seperator: Seperator) -> None:
        path = Path(self.path)
        filename = path.name
        format = path.suffix

        # get datetime
        parts = filename.split(seperator.value)
        datetime_seperator = datetime_parser.seperator

        datetime_is_continuous = (datetime_seperator == Seperator.EMPTY) or (datetime_seperator != seperator)
        if datetime_parser.from_beginning:
            if datetime_is_continuous:
                datetime_string = parts[0]
            else:
                datetime_string = datetime_seperator.value.join((parts[0], parts[1]))
        else:
            if datetime_is_continuous:
                datetime_string = parts[-1]
            else:
                datetime_string = datetime_seperator.value.join((parts[-2], parts[-1]))

        starttime = datetime_parser.str_to_datetime(datetime_string)

        meta = self.metadata_videofile
        meta.path = path
        meta.filename = filename
        meta.format = format
        meta.starttime = starttime

    def metadata_from_sidecarfile(self) -> None:
        pass

AttributeError: Cannot overwrite NamedTuple attribute __slots__

In [106]:
class Metadata(NamedTuple):
    path: str = None
    filename: str = None
    format: str = None
    starttime: datetime = None
    endtime: datetime = None
    fps: float = None
    width: int = None
    height: int = None
    format: str = None
    recording_token: str = None
    recordingblock_token: str = None
    status: str = None

In [109]:
m = Metadata()
m.path

In [99]:
from pathlib import Path

p = Path(r"N:\Mitarbeiter\Florian\anaconda3\WBKamera\WBKameraTestvids\18_Ichenheim\20180915\18\20180915_183000_8E51_00408CC568E0\20180916_05\20180916_055503_DF5A_00408CC568E0.mkv")
m = DatetimeParser(DatePattern.YYYYMMDD, TimePattern.HHmmss, Seperator.UNDERSCORE, from_beginning=True)

vf = VideoFile(p, Seperator.UNDERSCORE, m)
vf.metadata_from_videofile()

datetime.datetime(2018, 9, 16, 5, 55, 3)

In [118]:
import dateutil

datestring = meta['format']['tags']['creation_time']
dateutil.parser.isoparse(datestring)

datetime.datetime(2018, 9, 16, 3, 55, 3, tzinfo=tzutc())

In [133]:
import ffmpeg
import dateutil
import datetime

meta = ffmpeg.probe(p)
width = int(meta['streams'][0]['width'])
height = int(meta['streams'][0]['height'])
fps = int(meta['streams'][0]['avg_frame_rate'].split('/')[0])
duration = float(meta['format']['duration'])
starttime = dateutil.parser.isoparse(meta['format']['tags']['creation_time'])
endtime = starttime + datetime.timedelta(seconds=duration)
print(width, height, fps, starttime, endtime, duration)

768 576 30 2018-09-16 03:55:03+00:00 2018-09-16 04:00:03.088925+00:00 300.088925


In [132]:
starttime + datetime.timedelta(seconds=duration)

datetime.datetime(2018, 9, 16, 4, 0, 3, 88925, tzinfo=tzutc())

In [84]:
m = DatetimeParser(DatePattern.YYYYMMDD, TimePattern.HHmmss, Seperator.UNDERSCORE, from_beginning=True)
m.str_to_datetime('20180916_055503')

datetime.datetime(2018, 9, 16, 5, 55, 3)

In [77]:
m.seperator == Seperator.UNDERSCORE

True

In [151]:
from pathlib import Path

p = Path(r"N:\Mitarbeiter\Florian\anaconda3\WBKamera\WBKameraTestvids\18_Ichenheim\20180915\18\20180915_183000_8E51_00408CC568E0\20180916_05\20180916_055503_DF5A_00408CC568E0.mkv")
filename = p.stem

xml = p.with_suffix('.xml')
xml

WindowsPath('N:/Mitarbeiter/Florian/anaconda3/WBKamera/WBKameraTestvids/18_Ichenheim/20180915/18/20180915_183000_8E51_00408CC568E0/20180916_05/20180916_055503_DF5A_00408CC568E0.xml')

In [185]:
from typing import Optional
from datetime import datetime

class Metadata:
    __slots__ = ('starttime', 'stoptime', 'fps', 'width', 'height', 'status')
    aliases = {
        'endtime': 'stoptime'
    }

    def __init__(
            self,
            starttime: Optional[datetime] = None,
            stoptime: Optional[datetime] = None,
            fps: Optional[float] = None,
            width: Optional[int] = None,
            height: Optional[int] = None,
            status: Optional[str] = None
        ):

        self.starttime = starttime
        self.stoptime = stoptime
        self.fps = fps
        self.width = width
        self.height = height
        self.status = status

    def __setattr__(self, name, value):
        name = self.aliases.get(name, name)
        object.__setattr__(self, name, value)

    def __getattr__(self, name):
        if name == 'aliases':
            raise AttributeError  # http://nedbatchelder.com/blog/201010/surprising_getattr_recursion.html
        name = self.aliases.get(name, name)
        return object.__getattribute__(self, name)

In [246]:
projectfolder = Path(r"N:\Mitarbeiter\Florian\projects\CamTrapPy\tests\projectstructure\Projekt1")

from glob import glob
glob(str(projectfolder) + '/*/*/')

projectfolder = Path(r"N:\Projekte\BASt_Wirksamkeit_Wildwarnanlagen_1754")

locationfolders_subpath = 'Daten\Feldarbeit'
locationfolders = [Path(path) for path in glob(str(projectfolder / locationfolders_subpath) + '/*/')]

videofolders_subpath = 'Waermebildkameras'
if videofolders_subpath is not None:
    videofolders = [path / videofolders_subpath for path in locationfolders]
else:
    videofolders = locationfolders
    
sessions = True
sessionfolders = {}
if sessions:
    for i, folder in enumerate(videofolders):
        paths = [Path(path) for path in glob(str(folder) + '/*/')]
        if paths:
            location = locationfolders[i].name
            sessionfolders[location] = paths

sublocations = True
sublocationfolders = {}
if sublocations:
    for location, paths in sessionfolders.items():
        sublocationfolders[location] = {}
        for path in paths:
            session = path.name
            paths = [Path(path) for path in glob(str(path) + '/*/')]
            sublocationfolders[location][session] = paths

In [247]:
sublocationfolders

{'Aglasterhausen B292': {'session 1': [WindowsPath('N:/Projekte/BASt_Wirksamkeit_Wildwarnanlagen_1754/Daten/Feldarbeit/Aglasterhausen B292/Waermebildkameras/session 1/auxfiles'),
   WindowsPath('N:/Projekte/BASt_Wirksamkeit_Wildwarnanlagen_1754/Daten/Feldarbeit/Aglasterhausen B292/Waermebildkameras/session 1/wwa_B292_W_1_2'),
   WindowsPath('N:/Projekte/BASt_Wirksamkeit_Wildwarnanlagen_1754/Daten/Feldarbeit/Aglasterhausen B292/Waermebildkameras/session 1/wwa_B292_W_2_1'),
   WindowsPath('N:/Projekte/BASt_Wirksamkeit_Wildwarnanlagen_1754/Daten/Feldarbeit/Aglasterhausen B292/Waermebildkameras/session 1/wwa_B292_W_2_2'),
   WindowsPath('N:/Projekte/BASt_Wirksamkeit_Wildwarnanlagen_1754/Daten/Feldarbeit/Aglasterhausen B292/Waermebildkameras/session 1/events'),
   WindowsPath('N:/Projekte/BASt_Wirksamkeit_Wildwarnanlagen_1754/Daten/Feldarbeit/Aglasterhausen B292/Waermebildkameras/session 1/wwa_B292_W_1_1')],
  '20210628_11 Aufhängung': [],
  'session 2': [WindowsPath('N:/Projekte/BASt_Wirks

In [184]:
import xml.etree.ElementTree as ET

def parse(xml):
    if not xml.is_file():
        return

    meta = Metadata()
    
    tree = ET.parse(xml)
    root = tree.getroot()
    for child in root:
        tag = child.tag.lower()
        if tag in Metadata.__slots__:
            setattr(meta, tag, child.text)
            
    return meta
        
meta = parse(xml)
meta.starttime

'2018-09-16T03:55:03.262206Z'

In [174]:
Metadata.__slots__

('path',
 'filename',
 'format',
 'starttime',
 'endtime',
 'fps',
 'width',
 'height',
 'format',
 'recording_token',
 'recordingblock_token',
 'status')